## 1. Data Cleaning

Separate Data (MSM and SM)

In [ ]:
import pandas as pd

file_path = "PCO.xlsx"
df = pd.read_excel(file_path)

print("Columns in your file:")

print(df.columns)

df["Headline"] = df["Headline"].fillna("")
df["Source"] = df["Source"].fillna("").str.lower()
sm_keywords = ["facebook", "twitter", "instagram", "bsky", "youtube", "blogs", "reddit", "tiktok"]

force_msm_sources = [

    "germanexinthephilppines", "juancrisostomo", "wordpress",

    "kwebanibarok", "majeca", "republikanews.org", "reyfortmedia"

]

def classify_row(row):

    source = row["Source"]

    headline = row["Headline"]

    if any(keyword in source for keyword in force_msm_sources):

        return "MSM"

    if headline.strip() == "":

        return "SM"

    if any(keyword in source for keyword in sm_keywords):

        return "SM"

    return "MSM"

df["Category"] = df.apply(classify_row, axis=1)

msm_df = df[df["Category"] == "MSM"]

sm_df = df[df["Category"] == "SM"]

msm_df.to_csv("MSM.csv", index=False, encoding="utf-8-sig")

sm_df.to_csv("SM.csv", index=False, encoding="utf-8-sig")

print(f"✅ Done! Saved {len(msm_df)} rows to MSM.csv and {len(sm_df)} rows to SM.csv")


## 2. Clean SM Data

In [ ]:
import pandas as pd
import re
import string
import re
from langdetect import detect, LangDetectException

file_path = "SM.csv"
df = pd.read_csv(file_path)

df_clean = df.drop_duplicates(subset=["Source Link"], keep="first")

df_clean = df_clean.dropna(subset=["Date", "Source", "Country"])
def fill_headline(row):

    if pd.isna(row["Headline"]) or str(row["Headline"]).strip() == "":

        if pd.notna(row.get("Opening Text")) and str(row.get("Opening Text")).strip() != "":

            return row["Opening Text"].strip()

    return row["Headline"]

df_clean["Headline"] = df_clean.apply(fill_headline, axis=1)

def parse_date(row):

    date_str = row["Date"] if pd.notnull(row["Date"]) else row.get("Alternate Date Format", None)

    try:

        parsed = pd.to_datetime(date_str, errors="coerce")

        return parsed.date() if pd.notnull(parsed) else pd.NaT
    except:

        return pd.NaT

df_clean.loc[:, "Clean Date"] = df_clean.apply(parse_date, axis=1)

df_clean = df_clean.dropna(subset=["Clean Date"])

df_clean.loc[:, "Clean Date"] = pd.to_datetime(df_clean["Clean Date"]).dt.strftime("%m-%d-%Y")
def normalize_source(source):

    s = str(source).strip().lower()

    s = re.sub(r"www\.", "", s)
    s = re.sub(r"\.com|\.ph|\.net|\.org", "", s)
    s = re.sub(r"\s+", " ", s)
    if "reddit" in s:

        return "Forums"

    return s.title()

df_clean.loc[:, "Source"] = df_clean["Source"].apply(normalize_source)

def normalize_headline(text):

    if pd.isnull(text):

        return ""

    text = text.lower()

    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)

    text = re.sub(r"\s+", " ", text).strip()

    return text

df_clean["headline_key"] = df_clean["Headline"].apply(normalize_headline)

df_clean = df_clean.drop_duplicates(subset=["headline_key"], keep="first")

df_clean = df_clean.drop(columns=["headline_key"])

for col in ["Opening Text", "Influencer", "Key Phrases", "Language", "Sentiment"]:

    if col in df_clean.columns:

        df_clean[col] = df_clean[col].fillna("Unknown")
        df_clean[col] = df_clean[col].replace(r"^\s*$", "Unknown", regex=True)
social_echo_cols = ["Twitter Social Echo", "Facebook Social Echo", "Reddit Social Echo"]

cols_to_fix = social_echo_cols + ["Reach"]

for col in cols_to_fix:

    if col in df_clean.columns:

        df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce").fillna(0).astype(int)

if all(col in df_clean.columns for col in social_echo_cols):

    df_clean["Social Echo Total"] = (

        df_clean["Twitter Social Echo"]

        + df_clean["Facebook Social Echo"]

        + df_clean["Reddit Social Echo"]

    ).astype(int)

elif "Social Echo Total" in df_clean.columns:

    df_clean["Social Echo Total"] = pd.to_numeric(df_clean["Social Echo Total"], errors="coerce").fillna(0).astype(int)

df_clean = df_clean[~df_clean["Headline"].str.contains("content from this publisher", case=False, na=False)]

df_clean = df_clean[~df_clean["Source Link"].str.contains("proquest", case=False, na=False)]

df_clean = df_clean[~df_clean["Headline"].str.contains("test", case=False, na=False)]

df_clean = df_clean[df_clean["Source Link"].str.strip() != ""]

if "Hit Sentence" in df_clean.columns:

    df_clean = df_clean[~df_clean["Hit Sentence"].str.contains(r"\[Courtesy:|\[Photo courtesy\]", case=False, na=False)]

cols_to_drop = [

    "Subregion", "Desktop Reach", "Mobile Reach",

    "National Viewership", "Engagement", "AVE", "Twitter Authority",

    "Tweet Id", "Twitter Id", "Twitter Client", "Twitter Screen Name",

    "User Profile Url", "Twitter Bio", "Twitter Followers", "Twitter Following",

    "Alternate Date Format", "Time", "State", "City", "Editorial Echo",

    "Views", "Estimated Views", "Likes", "Replies", "Retweets",

    "Comments", "Shares", "Reactions", "Threads", "Is Verified",

    "Parent URL", "Document Tags", "Document ID", "Custom Categories"

]

df_clean = df_clean.drop(columns=[c for c in cols_to_drop if c in df_clean.columns])

if "Country" in df_clean.columns:

    df_clean["Country"] = df_clean["Country"].astype(str).str.strip().str.title()

if "Headline" in df_clean.columns:

    df_clean["Headline"] = df_clean["Headline"].astype(str).str.strip()

    df_clean["Headline"] = df_clean["Headline"].replace(r"\s+", " ", regex=True)

    df_clean["Headline"] = df_clean["Headline"].str.title()
text_cols = ["Opening Text", "Hit Sentence"]

for col in text_cols:

    if col in df_clean.columns:

        df_clean[col] = df_clean[col].astype(str).str.strip()

        df_clean[col] = df_clean[col].replace(r"\s+", " ", regex=True)

        df_clean[col] = df_clean[col].str.capitalize()


if "Clean Date" in df_clean.columns:
    cols = ["Clean Date"] + [c for c in df_clean.columns if c != "Clean Date"]
    df_clean = df_clean[cols]

# addition for standardization and new conditions for cleaning - updated September 2025
def standardize_keyphrases(text):
    if pd.isna(text) or str(text).strip() == "":
        return "Unknown"
    text = str(text)
    parts = re.split(r'[;,|/•]+', text)
    normalized = []
    for p in parts:
        p = p.strip()
        if p == "":
            continue
        low = p.lower()
        if "pco" in low or "presidential communications" in low:
            normalized.append("Presidential Communications Office")
        elif "philippines" in low or "pilipinas" in low or "pilipino" in low:
            normalized.append("Philippines")
        else:
            normalized.append(p.title())
    seen = set()
    final_list = []
    for item in normalized:
        if item not in seen:
            final_list.append(item)
            seen.add(item)
    return ", ".join(final_list) if final_list else "Unknown"

if "Key Phrases" in df_clean.columns:
    df_clean["Key Phrases"] = df_clean["Key Phrases"].apply(standardize_keyphrases)

if "Keywords" in df_clean.columns:
    df_clean["Keywords"] = df_clean["Keywords"].apply(standardize_keyphrases)

def detect_language_for_country(text):
    if pd.isna(text) or str(text).strip() == "":
        return None
    try:
        return detect(text)
    except LangDetectException:
        return None

def map_language_to_country(lang_code):
    mapping = {
        "tl": "Philippines",
        "fil": "Philippines",
        "en": "United States",
        "ja": "Japan",
        "ko": "South Korea",
        "zh": "China",
        "fr": "France",
        "es": "Spain",
        "de": "Germany",
        "ms": "Malaysia",
        "id": "Indonesia",
    }
    return mapping.get(lang_code, None)

if "Country" in df_clean.columns and "Hit Sentence" in df_clean.columns:
    def fill_country(row):
        country = str(row["Country"]).strip().lower()
        if country == "" or country == "unknown" or country == "nan":
            lang_code = detect_language_for_country(row["Hit Sentence"])
            inferred_country = map_language_to_country(lang_code)
            return inferred_country if inferred_country else "Unknown"
        return row["Country"]

    df_clean["Country"] = df_clean.apply(fill_country, axis=1)



def detect_tl_or_en(text):
    if pd.isna(text) or str(text).strip() == "":
        return "English"
    try:
        lang_code = detect(str(text))
    except LangDetectException:
        return "English"

    if lang_code in ["tl", "fil"]:
        return "Tagalog"
    else:
        return "English"

if "Language" in df_clean.columns and "Hit Sentence" in df_clean.columns:
    def fill_language(row):
        lang = str(row["Language"]).strip().lower()
        if lang == "" or lang == "unknown" or lang == "nan":
            return detect_tl_or_en(row["Hit Sentence"])
        return row["Language"]

    df_clean["Language"] = df_clean.apply(fill_language, axis=1)


output_path = "SM_cleaned.xlsx"
df_clean.to_excel(output_path, index=False, engine="openpyxl")

print("Cleaning complete!")
print(f"Remaining rows: {len(df_clean)}")


## 3. Sentiment Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob

In [ ]:
df = pd.read_excel('SM_cleaned.xlsx')

print(f"Initial dataset shape: {df.shape}")

print(f"\nColumn names:\n{df.columns.tolist()}")

In [ ]:
import pandas as pd

file_path = "SM_cleaned.xlsx"
df_cleaned = pd.read_excel(file_path)

print("✅ Original Sentiment Distribution from SM_cleaned.xlsx:\n")

print(df_cleaned['Sentiment'].value_counts(dropna=False))

rows_with_sentiment = df_cleaned['Sentiment'].notna().sum()
rows_without_sentiment = df_cleaned['Sentiment'].isna().sum()

print(f"\nRows with sentiment: {rows_with_sentiment}")
print(f"Rows without sentiment: {rows_without_sentiment}")


In [ ]:
import pandas as pd
from textblob import TextBlob

file_path = "SM_cleaned.xlsx"
df_cleaned = pd.read_excel(file_path)

def get_new_sentiment(text):

    if pd.isnull(text):

        return 'Neutral'

    polarity = TextBlob(str(text)).sentiment.polarity

    if polarity > 0.1:

        return 'Positive'

    elif polarity < -0.1:

        return 'Negative'

    else:

        return 'Neutral'

df['New_Sentiment'] = df['Headline'].apply(get_new_sentiment)

print('New Sentiment Distribution:')

print(df['New_Sentiment'].value_counts())

print()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = "SM_cleaned.xlsx"
df_cleaned = pd.read_excel(file_path)

sentiment_counts = df_cleaned['Sentiment'].value_counts()

color_map = {
    "Positive": "green",
    "Negative": "red",
    "Neutral": "gray",
    "Unknown": "orange"
}
colors = [color_map.get(s, "blue") for s in sentiment_counts.index]

plt.figure(figsize=(8, 5))
ax = sentiment_counts.plot(kind='bar', color=colors)

plt.title("Original Sentiment Distribution", fontsize=14, fontweight='bold')
plt.xlabel("Sentiment", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()

for i, v in enumerate(sentiment_counts):
    ax.text(i, v + max(sentiment_counts) * 0.01, str(v),
            ha='center', va='bottom', fontweight='bold')

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = "SM_cleaned.xlsx"
df_cleaned = pd.read_excel(file_path)

if "New_Sentiment" not in df_cleaned.columns:
    from textblob import TextBlob

    def get_new_sentiment(text):
        if pd.isnull(text):
            return 'Neutral'
        polarity = TextBlob(str(text)).sentiment.polarity
        if polarity > 0.1:
            return 'Positive'
        elif polarity < -0.1:
            return 'Negative'
        else:
            return 'Neutral'

    df_cleaned['New_Sentiment'] = df_cleaned['Headline'].apply(get_new_sentiment)

new_sentiment_counts = df_cleaned['New_Sentiment'].value_counts()

color_map = {
    "Positive": "green",
    "Negative": "red",
    "Neutral": "gray",
    "Unknown": "orange"
}
colors = [color_map.get(s, "blue") for s in new_sentiment_counts.index]

plt.figure(figsize=(8, 5))
ax = new_sentiment_counts.plot(kind='bar', color=colors)

plt.title("New Sentiment Distribution", fontsize=14, fontweight='bold')
plt.xlabel("Sentiment", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()

for i, v in enumerate(new_sentiment_counts):
    ax.text(i, v + max(new_sentiment_counts) * 0.01, str(v),
            ha='center', va='bottom', fontweight='bold')

plt.show()


In [ ]:
import pandas as pd
from textblob import TextBlob

file_path = "SM_cleaned.xlsx"
df_cleaned = pd.read_excel(file_path)

if "New_Sentiment" not in df_cleaned.columns:
    def get_new_sentiment(text):
        if pd.isnull(text):
            return 'Neutral'
        polarity = TextBlob(str(text)).sentiment.polarity
        if polarity > 0.1:
            return 'Positive'
        elif polarity < -0.1:
            return 'Negative'
        else:
            return 'Neutral'

    df_cleaned['New_Sentiment'] = df_cleaned['Headline'].apply(get_new_sentiment)

diff_sentiment_samples = df_cleaned[df_cleaned['Sentiment'] != df_cleaned['New_Sentiment']]

sample_count = min(5, len(diff_sentiment_samples))
diff_sentiment_samples = diff_sentiment_samples.sample(n=sample_count, random_state=42)

print(f"🧪 Showing {sample_count} examples where the sentiment changed:\n")
for idx, row in diff_sentiment_samples.iterrows():
    print(f"Headline: {row['Headline']}")
    print(f"Original Sentiment: {row['Sentiment']}")
    print(f"New Sentiment: {row['New_Sentiment']}")
    print('-' * 60)


In [ ]:
output_file = 'sentiment_analysis_results.xlsx'

df.to_excel(output_file, index=False)

print(f"Excel file '{output_file}' has been created successfully!")

## 4. Challenge 3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from wordcloud import WordCloud
from collections import Counter
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob
from wordcloud import WordCloud
from collections import Counter
import re

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)

df = pd.read_excel('sentiment_analysis_results.xlsx')
print(f"Initial dataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")

In [ ]:
file_path = "sentiment_analysis_results.xlsx"
df_results = pd.read_excel(file_path)

print("Dataset Info:")
df_results.info()

print("\n" + "="*50 + "\n")

print("First 5 rows:")
print(df_results.head())


In [ ]:
file_path = "sentiment_analysis_results.xlsx"
df_results = pd.read_excel(file_path)

daily_volume = (
    df.groupby('Clean Date')
      .size()
      .reset_index(name='Story_Count')
      .sort_values('Clean Date')
)

fig, ax = plt.subplots(figsize=(15, 5))

ax.plot(
    daily_volume['Clean Date'],
    daily_volume['Story_Count'],
    color='purple',
    linewidth=2,
    marker='o',
    markersize=4,
    alpha=0.8,
    label='Story Volume'
)

ax.set_title("Daily Story Volume – Trendline", fontsize=16, fontweight='bold')
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Number of Stories", fontsize=12)

ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
file_path = "sentiment_analysis_results.xlsx"
df_results = pd.read_excel(file_path)

if "New_Sentiment" not in df_results.columns:
    raise ValueError("⚠️ 'New_Sentiment' column not found in sentiment_analysis_results.xlsx")

sentiment_counts = df_results['New_Sentiment'].value_counts()

color_map = {
    "Positive": "green",
    "Negative": "red",
    "Neutral": "gray",
    "Unknown": "orange"
}

colors = [color_map.get(label, "blue") for label in sentiment_counts.index]

fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(
    sentiment_counts.values,
    labels=sentiment_counts.index,
    autopct='%1.1f%%',
    colors=colors,
    startangle=140
)

ax.set_title('Overall Sentiment Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
file_path = "sentiment_analysis_results.xlsx"
df_results = pd.read_excel(file_path)

top_sources = df['Source'].value_counts().head(10)

plt.figure(figsize=(12, 8))
top_sources.sort_values().plot(kind='barh', color='skyblue')
plt.title('Top 10 Sources by Story Count', fontsize=14)
plt.xlabel('Number of Stories')
plt.ylabel('Source')
plt.tight_layout()
plt.show()

In [ ]:
file_path = "sentiment_analysis_results.xlsx"
df_results = pd.read_excel(file_path)

top_countries = df['Country'].value_counts().head(10)

plt.figure(figsize=(12, 8))
top_countries.sort_values().plot(kind='barh', color='coral')
plt.title('Top 10 Countries by Story Count', fontsize=14)
plt.xlabel('Number of Stories')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = 'sentiment_analysis_results.xlsx.'
df = pd.read_excel(file_path, sheet_name="Sheet1")

df['Clean Date'] = pd.to_datetime(df['Clean Date'], errors='coerce')

df['Reach'] = pd.to_numeric(df['Reach'], errors='coerce').fillna(0)

daily_reach = df.groupby('Clean Date')['Reach'].sum().reset_index()

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(daily_reach['Clean Date'], daily_reach['Reach'], linewidth=2, color='purple')
ax.set_title('Daily Reach - Trendline', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Reach')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = 'sentiment_analysis_results.xlsx'
df = pd.read_excel(file_path, sheet_name="Sheet1")

df['Total_Social_Echo'] = pd.to_numeric(df['Social Echo Total'], errors='coerce').fillna(0)

df['Clean Date'] = pd.to_datetime(df['Clean Date'], errors='coerce').dt.strftime('%Y-%m-%d')

daily_social_echo = df.groupby('Clean Date')['Total_Social_Echo'].sum().reset_index()
daily_social_echo = daily_social_echo.sort_values('Clean Date')

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(
    daily_social_echo['Clean Date'],
    daily_social_echo['Total_Social_Echo'],
    linewidth=2,
    color='purple'
)

ax.set_title('Daily Social Echo - Trendline', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Total Social Echo')
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 5. Challenge 4

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from wordcloud import WordCloud
from collections import Counter
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

Code to Identify the Top 10 Keywords via Keywords Column

In [ ]:
df = pd.read_excel('sentiment_analysis_results.xlsx.')

print("Analyzing Keywords...")

all_keywords = []

for keywords in df['Keywords'].dropna():

    if ',' in str(keywords):

        keyword_list = str(keywords).split(',')

    elif ';' in str(keywords):

        keyword_list = str(keywords).split(';')

    else:

        keyword_list = [str(keywords)]

    all_keywords.extend([k.strip() for k in keyword_list if k.strip()])

keyword_counts = Counter(all_keywords)

top_keywords = keyword_counts.most_common(10)

print(f"Total unique keywords: {len(keyword_counts)}")

print(f"Total keyword occurrences: {len(all_keywords)}")

print("\nTop 10 Keywords:")

for keyword, count in top_keywords:

    print(f"{keyword}: {count}")

Code to Identify the Top 10 Phrases via Key Phrases Column

In [ ]:
df = pd.read_excel('sentiment_analysis_results.xlsx.')

print("\nAnalyzing Key Phrases...")

all_phrases = []

for phrases in df['Key Phrases'].dropna():

    if ',' in str(phrases):

        phrase_list = str(phrases).split(',')

    elif ';' in str(phrases):

        phrase_list = str(phrases).split(';')

    else:

        phrase_list = [str(phrases)]

    all_phrases.extend([p.strip() for p in phrase_list if p.strip()])

phrase_counts = Counter(all_phrases)

top_phrases = phrase_counts.most_common(10)

print(f"Total unique phrases: {len(phrase_counts)}")

print(f"Total phrase occurrences: {len(all_phrases)}")

print("\nTop 10 Key Phrases:")

for phrase, count in top_phrases:

    print(f"{phrase}: {count}")

Code to Identify the Top 10 Keywords via Keywords Column (VISUALIZATION)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_excel("sentiment_analysis_results.xlsx")

keywords_df = pd.DataFrame(top_keywords, columns=['Keyword', 'Count'])
keywords_df['Keyword'] = keywords_df['Keyword'].str.replace(' ', '\n')

fig, ax = plt.subplots(1, 1, figsize=(12, 5))
bars = keywords_df.plot(
    x='Keyword', 
    y='Count', 
    kind='bar', 
    ax=ax, 
    color='#2D3289', 
    legend=False
)

ax.set_facecolor('#FFFFFF')
ax.set_title('Top Keywords', fontsize=12, color='#2D3289', fontweight='bold')
ax.set_xlabel('Keyword', fontsize=10, color='#2D3289')
ax.set_ylabel('Frequency', fontsize=10, color='#2D3289')
ax.tick_params(rotation=0, axis='x', labelsize=8)
ax.tick_params(axis='y', labelsize=8)

for patch in ax.patches:
    height = patch.get_height()
    ax.text(
        patch.get_x() + patch.get_width() / 2,
        height + max(keywords_df['Count']) * 0.01,  
        f"{int(height)}", 
        ha='center', 
        va='bottom', 
        fontsize=9, 
        fontweight='bold',
        color='#2D3289'
    )

plt.tight_layout()
plt.show()


Code to Identify the Top 10 Phrases via Key Phrases Column (VISUALIZATION)

In [ ]:
df = pd.read_excel("sentiment_analysis_results.xlsx")

fig, ax = plt.subplots(1, 1, figsize=(12, 5))

phrases_df = pd.DataFrame(top_phrases, columns=['Phrase', 'Count'])
phrases_df = phrases_df[~phrases_df['Phrase'].isin(['Uncategorized', 'Unknown', 'N/A'])]

phrases_df = phrases_df.head(10)

phrases_df['Phrase'] = phrases_df['Phrase'].str.replace(' ', '\n')

# Plot
phrases_df.plot(x='Phrase', y='Count', kind='bar', ax=ax, color='#2D3289')
ax.set_title('Top 10 Key Phrases (excluding Uncategorized/Unknown)', fontsize=12, color='#2D3289', fontweight='bold')
ax.set_xlabel('Key Phrase', fontsize=10, color='#2D3289')
ax.set_ylabel('Frequency', fontsize=10, color='#2D3289')
ax.tick_params(rotation=0, axis='x', labelsize=8)
ax.tick_params(axis='y', labelsize=8)
plt.tight_layout()
plt.show()


Code to Identify the Top 10 Keywords in the Headline via Headline Column with Visualization

In [ ]:
df = pd.read_excel("sentiment_analysis_results.xlsx")

print("Analyzing Keywords...")
all_keywords = []
for keywords in df['Headline'].dropna():
    if ',' in str(keywords):
        keyword_list = str(keywords).split(',')
    elif ';' in str(keywords):
        keyword_list = str(keywords).split(';')
    else:
        keyword_list = [str(keywords)]
    all_keywords.extend([k.strip() for k in keyword_list if k.strip()])
keyword_counts = Counter(all_keywords)
top_keywords = keyword_counts.most_common(10)
print(f"Total unique keywords in Headline: {len(keyword_counts)}")
print(f"Total keyword occurrences in Headline: {len(all_keywords)}")
print("\nTop 10 Keywords in Headline:")
for keyword, count in top_keywords:
    print(f"{keyword}: {count}")
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
keywords_df = pd.DataFrame(top_keywords, columns=['Keyword', 'Count'])
keywords_df['Keyword'] = keywords_df['Keyword'].str.replace(' ', '\n')
keywords_df.plot(x='Keyword', y='Count', kind='bar', ax=ax, color='#2D3289')
ax.set_facecolor('#FFFFFF')
ax.set_title('Top 10 Keywords in Headline', fontsize=12, color='#2D3289', fontweight='bold')
ax.set_xlabel('Keyword', fontsize=10, color='#2D3289')
ax.set_ylabel('Frequency', fontsize=10, color='#2D3289')
ax.tick_params(rotation=0, axis='x', labelsize=8,)
ax.tick_params(axis='y', labelsize=8)
plt.tight_layout()
plt.show()

Code to Identify the Top 3 Keywords within the Top 5 Sources via Keywords & Sources Column

In [ ]:
df = pd.read_excel("sentiment_analysis_results.xlsx")

print("\nKeyword Usage by Source:")

source_keywords = {}

for idx, row in df.iterrows():

    source = row['Source']

    keywords_str = row['Keywords']

    if pd.notna(keywords_str) and pd.notna(source):

        if ',' in str(keywords_str):

            keywords = [k.strip() for k in str(keywords_str).split(',')]

        elif ';' in str(keywords_str):

            keywords = [k.strip() for k in str(keywords_str).split(';')]

        else:

            keywords = [str(keywords_str).strip()]

        if source not in source_keywords:

            source_keywords[source] = []

        source_keywords[source].extend(keywords)

top_5_sources = df['Source'].value_counts().head(5).index

print("\nTop Keywords by Top 5 Sources:")

for source in top_5_sources:

    if source in source_keywords:

        source_keyword_counts = Counter(source_keywords[source])

        top_3_keywords = source_keyword_counts.most_common(3)

        print(f"\n{source}:")

        for keyword, count in top_3_keywords:

            print(f"  - {keyword}: {count}")

Code to Identify the Top 3 Key Phrases within the Top 5 Sources via Key Phrases & Sources Column

In [ ]:
df = pd.read_excel("sentiment_analysis_results.xlsx")

print("\nTop Phrases by Top 5 Sources:")

top_5_sources = df['Source'].value_counts().head(5).index

source_phrases = {}

for idx, row in df.iterrows():

    source = row['Source']

    phrases_str = row['Key Phrases']

    if pd.notna(source) and source in top_5_sources and pd.notna(phrases_str):

        if ',' in phrases_str:

            phrases = [p.strip() for p in phrases_str.split(',')]

        elif ';' in phrases_str:

            phrases = [p.strip() for p in phrases_str.split(';')]

        else:

            phrases = [phrases_str.strip()]

        if source not in source_phrases:

            source_phrases[source] = []

        source_phrases[source].extend(phrases)

for source in top_5_sources:

    phrases = source_phrases.get(source, [])

    phrase_counts = Counter(phrases)

    top_phrases = phrase_counts.most_common(3)  

    print(f"\n{source}:")

    for phrase, count in top_phrases:

        print(f"  - {phrase}: {count}")

In [ ]:
df1 = pd.read_excel('SM_cleaned.xlsx')
df2 = pd.read_excel('sentiment_analysis_results.xlsx') 

Top Keywords by Sentiment

In [ ]:
df = pd.read_excel("sentiment_analysis_results.xlsx")

print("\nTop Keywords by Sentiment:")

sentiments = df2['New_Sentiment'].dropna().unique()

sentiment_keywords = {}

for idx, row in df2.iterrows():
    sentiment = row['New_Sentiment']
    keywords_str = row['Keywords']
    
    if pd.notna(sentiment) and pd.notna(keywords_str):
        if ',' in keywords_str:
            keywords = [p.strip() for p in keywords_str.split(',')]
        elif ';' in keywords_str:
            keywords = [p.strip() for p in keywords_str.split(';')]
        else:
            keywords = [keywords_str.strip()]
        
        if sentiment not in sentiment_keywords:
            sentiment_keywords[sentiment] = []
        sentiment_keywords[sentiment].extend(keywords)

for sentiment in sentiments:
    keywords = sentiment_keywords.get(sentiment, [])
    keyword_counts = Counter(keywords)
    top_keywords = keyword_counts.most_common(3) 
    
    print(f"\n{sentiment}:")
    for keyword, count in top_keywords:
        print(f"  - {keyword}: {count}")

Top Key Phrases by Sentiment

In [ ]:
from collections import Counter
import pandas as pd

df = pd.read_excel("sentiment_analysis_results.xlsx")

print("\nTop Key Phrases by Sentiment:")

sentiments = df2['New_Sentiment'].dropna().unique()

sentiment_phrases = {}

for idx, row in df2.iterrows():
    sentiment = row['New_Sentiment']
    phrases_str = row['Key Phrases']
    
    if pd.notna(sentiment) and pd.notna(phrases_str):
        if ',' in phrases_str:
            phrases = [p.strip() for p in phrases_str.split(',')]
        elif ';' in phrases_str:
            phrases = [p.strip() for p in phrases_str.split(';')]
        else:
            phrases = [phrases_str.strip()]
        
        if sentiment not in sentiment_phrases:
            sentiment_phrases[sentiment] = []
        sentiment_phrases[sentiment].extend(phrases)

for sentiment in sentiments:
    phrases = sentiment_phrases.get(sentiment, [])
    phrase_counts = Counter(phrases)
    top_phrases = phrase_counts.most_common(3)
    
    print(f"\n{sentiment}:")
    for phrase, count in top_phrases:
        print(f"  - {phrase}: {count}")

## 6. Challenge 5

Word cloud from Key Phrases, Keywords, and extracted topics

In [ ]:
df = pd.read_excel("sentiment_analysis_results.xlsx")

all_text_keywords = ' '.join(map(str, all_keywords))

try:
    all_text_phrases = ' '.join(map(str, all_phrases))
except NameError:
    all_text_phrases = ''

combined_text = (all_text_keywords + ' ' + all_text_phrases).strip()

if combined_text:
    plt.figure(figsize=(15, 8))
    wordcloud = WordCloud(width=800, height=400, background_color='white', 
                         colormap='viridis', max_words=100).generate(combined_text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Overall Word Cloud - All Keywords and Phrases', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("No data available to generate a word cloud.")


Generate separate clouds for different sentiment categories

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

df = pd.read_excel("sentiment_analysis_results.xlsx")

sentiment_column = "New_Sentiment" if "New_Sentiment" in df.columns else "Sentiment"

colormaps = {
    "Positive": "Greens",
    "Negative": "Reds",
    "Neutral": "Blues"
}

sentiments = ["Positive", "Negative", "Neutral"]

fig, axes = plt.subplots(1, 3, figsize=(20, 8))

for ax, sentiment in zip(axes, sentiments):
    sentiment_df = df[df[sentiment_column].str.lower() == sentiment.lower()]

    all_text_keywords = ' '.join(map(str, sentiment_df["Keywords"]))
    all_text_phrases = ' '.join(map(str, sentiment_df["Key Phrases"]))
    combined_text = (all_text_keywords + ' ' + all_text_phrases).strip()

    if combined_text:
        wordcloud = WordCloud(width=800, height=400, background_color='white',
                              colormap=colormaps[sentiment], max_words=100).generate(combined_text)
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.set_title(f'{sentiment} Sentiment Word Cloud', fontsize=14, fontweight='bold')
    else:
        ax.text(0.5, 0.5, f"No data for {sentiment}", ha='center', va='center', fontsize=14)

    ax.axis('off')

plt.tight_layout()
plt.show()


Create time-based word clouds (monthly or quarterly)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from wordcloud import WordCloud

In [ ]:
df = pd.read_excel('sentiment_analysis_results.xlsx')
print(f"Initial dataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")

In [ ]:
if not pd.api.types.is_datetime64_any_dtype(df['Date']):
    df['Date'] = pd.to_datetime(df['Date'])

def get_week_of_month(date):
    first_day = date.replace(day=1)
    dom = date.day
    adjusted_dom = dom + first_day.weekday()
    return int(np.ceil(adjusted_dom/7.0))

df.insert(df.columns.get_loc('Date') + 1, 'Week', df['Date'].apply(get_week_of_month))

print(f"New dataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")

print("\nFirst five rows:")
print(df.head())

In [ ]:
all_keywords = df['Keywords'].dropna().astype(str).tolist()
all_phrases = df['Key Phrases'].dropna().astype(str).tolist()
all_text_keywords = ' '.join(all_keywords)
all_text_phrases = ' '.join(all_phrases)
combined_text = all_text_keywords + ' ' + all_text_phrases

In [ ]:
weeks = sorted(df['Week'].unique())

for week in weeks:
    week_df = df[df['Week'] == week]
    all_keywords = week_df['Keywords'].dropna().astype(str).tolist()
    all_phrases = week_df['Key Phrases'].dropna().astype(str).tolist()
    combined_text = ' '.join(all_keywords) + ' ' + ' '.join(all_phrases)
    if combined_text.strip():
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(combined_text)
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Word Cloud for Week {week}')
        plt.show()
    else:
        print(f'No data for week {week}')

In [ ]:
output_file = 'FINAL_SM.xlsx'

df.to_excel(output_file, index=False)

print(f"Excel file '{output_file}' has been created successfully!")